In [9]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [107]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[5])

In [109]:
df_old = df_old.sort_values(["bundesland", "datum"])

df_old.drop_duplicates(inplace=True)

df_old = df_old.reset_index().drop("index",axis = 1)

In [120]:
today = pd.Timestamp(date.today())

In [121]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [122]:
page = requests.get(URL)

In [123]:
soup = BeautifulSoup(page.content, 'html.parser')

In [124]:
results = soup.find_all("tr")

In [125]:
bundesland = []
faelle = []
tod = []
neue_faelle=[]
faelle_per_capita=[]
for idx,elm in enumerate(results):
    if idx > 1:
        bundesland.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])))
        faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])))
        neue_faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[2])))
        faelle_per_capita.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[3])))
        tod.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[4])))
    else:
        pass

In [126]:
faelle = [int(elm.replace(".","")) for elm in faelle]
tod = [int(elm.replace(".","")) for elm in tod]
neue_faelle = [int(elm.strip("+").replace(".","")) for elm in neue_faelle]
faelle_per_capita = [float(elm.replace(",",".")) for elm in faelle_per_capita]

In [127]:
faelle_per_capita

[13.4,
 8.5,
 9.2,
 2.9,
 8.3,
 16.8,
 6.0,
 2.8,
 4.1,
 11.7,
 10.8,
 7.6,
 4.5,
 2.6,
 4.4,
 2.4,
 8.6]

In [128]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [129]:
df=pd.DataFrame(data=[bundesland,faelle,tod,neue_faelle,faelle_per_capita]).T
    

In [130]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [131]:
df.columns = ["bundesland","faelle","tod","neue_faelle","faelle_per_capita","lat","long","datum"]

In [132]:
#df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [133]:
df.datum = pd.to_datetime(df.datum)

In [134]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
lat2=[]
long2=[]
for elm in df.bundesland.values:
    lat2.append(lat[elm])
    long2.append(long[elm])

In [135]:
df["lat"] = lat2
df["long"] = long2

In [136]:
df

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,1479,2,417,13.4,48.50,9.00,2020-03-17
1,Bayern,1109,4,144,8.5,48.90,11.40,2020-03-17
2,Berlin,345,0,45,9.2,52.50,13.40,2020-03-17
3,Brandenburg,73,0,8,2.9,52.00,13.87,2020-03-17
4,Bremen,57,0,1,8.3,53.10,8.80,2020-03-17
5,Hamburg,310,0,55,16.8,53.50,10.00,2020-03-17
6,Hessen,373,0,132,6,50.70,9.00,2020-03-17
7,Mecklenburg-Vorpommern,45,0,9,2.8,53.80,12.50,2020-03-17
8,Niedersachsen,325,0,53,4.1,52.57,10.00,2020-03-17
9,Nordrhein-Westfalen,2105,6,564,11.7,51.60,7.50,2020-03-17


In [137]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")
df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")
df.neue_faelle = df.neue_faelle.astype("int64")
df.faelle_per_capita = df.faelle_per_capita.astype("float64")

In [138]:
pd.concat([df_old,df]).reset_index().drop("index",axis=1)

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,10,0,0,0.1,48.50,9.00,2020-02-28
1,Baden-Württemberg,14,0,4,0.1,48.50,9.00,2020-02-29
2,Baden-Württemberg,15,0,1,0.1,48.50,9.00,2020-03-01
3,Baden-Württemberg,19,0,4,0.2,48.50,9.00,2020-03-02
4,Baden-Württemberg,26,0,7,0.2,48.50,9.00,2020-03-03
5,Baden-Württemberg,44,0,18,0.4,48.50,9.00,2020-03-04
6,Baden-Württemberg,65,0,21,0.6,48.50,9.00,2020-03-05
7,Baden-Württemberg,91,0,26,0.8,48.50,9.00,2020-03-06
8,Baden-Württemberg,116,0,25,1.0,48.50,9.00,2020-03-07
9,Baden-Württemberg,182,0,66,1.6,48.50,9.00,2020-03-08


In [139]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [140]:
df[df["bundesland"]=="Bayern"]

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
18,Bayern,15,0,0,0.1,48.9,11.4,2020-02-28
19,Bayern,15,0,0,0.1,48.9,11.4,2020-02-29
20,Bayern,19,0,4,0.1,48.9,11.4,2020-03-01
21,Bayern,25,0,6,0.2,48.9,11.4,2020-03-02
22,Bayern,35,0,10,0.3,48.9,11.4,2020-03-03
23,Bayern,48,0,13,0.4,48.9,11.4,2020-03-04
24,Bayern,52,0,4,0.4,48.9,11.4,2020-03-05
25,Bayern,79,0,27,0.6,48.9,11.4,2020-03-06
26,Bayern,117,0,38,0.9,48.9,11.4,2020-03-07
27,Bayern,148,0,31,1.1,48.9,11.4,2020-03-08


In [141]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [142]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)